In [ ]:
import sys
sys.path.append('..')

from pathlib import Path
from ultralytics import YOLO
import cv2

from tracker import norfairDev
from utils import utils

# NorfairDev Example

In [ ]:
# inherited norfairDevTracker
class ExampleTracker(norfairDev.norfairDevTracker):
    def __init__(self, distance_function, distance_threshold, hit_counter_max=15, initialization_delay=None, pointwise_hit_counter_max=4, detection_threshold=0, filter_factory = None, past_detections_length=4, reid_distance_function=None, reid_distance_threshold=0, reid_hit_counter_max=None):
        super().__init__(distance_function, distance_threshold, hit_counter_max, initialization_delay, pointwise_hit_counter_max, detection_threshold, filter_factory, past_detections_length, reid_distance_function, reid_distance_threshold, reid_hit_counter_max)

In [ ]:
# inherited norfairDevTracker
class ExampleTrackedObject(norfairDev.norfairDevTrackedObject):
    def __init__(self, obj_factory, initial_detection, hit_counter_max, initialization_delay, pointwise_hit_counter_max, detection_threshold, period, filter_factory, past_detections_length, reid_hit_counter_max, coord_transformations=None):
        super().__init__(obj_factory, initial_detection, hit_counter_max, initialization_delay, pointwise_hit_counter_max, detection_threshold, period, filter_factory, past_detections_length, reid_hit_counter_max, coord_transformations)

In [ ]:
# initialization tracker
import numpy as np
example_tracker = ExampleTracker(
    initialization_delay=1,
    distance_function="euclidean",
    distance_threshold=100,
).set_tracker(
    custom_tracked_object=ExampleTrackedObject, # custom TrackedObject
    color_mapping_keys={
        'roi':(0,255,0),
        'roni':(0,0,255)
    },
    roni1 = np.array(
        [[0.05943691345151199, 0.4044526901669759], [0.03441084462982273, 0.7161410018552876], [0.0364963503649635, 0.9981447124304267], [0.7935349322210636, 0.9981447124304267], [0.2846715328467153, 0.849721706864564], [0.23670490093847757, 0.8645640074211502], [0.21793534932221065, 0.8181818181818182]]
    ),
    roni2 = np.array(
        [[0.8060479666319083, 0.3339517625231911], [0.748696558915537, 0.31539888682745826], [0.6934306569343066, 0.601113172541744], [0.9979144942648592, 0.7532467532467533], [0.9874869655891554, 0.38961038961038963]]
    )
)

In [ ]:
video_path = Path(r'C:\Users\articulusaitraining\projects\ptg\punthai\videos\29\D01_20250529063837_re-encode.mp4')
model_path = Path(r'C:\Users\articulusaitraining\projects\ptg\punthai\weights\v1x_PunthaiChaengWatthana_freeze10.pt')
model=YOLO(model_path)

In [ ]:
cap=cv2.VideoCapture(video_path)

draw = norfairDev.BaseDrawer()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret: 
            result = model.predict(frame, verbose=False)[0]
            points = result.boxes.xywh.cpu()[:, :2]
            boxes = result.boxes.xyxy.cpu()
            example_tracker.update_detections(points=points, data=utils.conv2dataDict(boxes=boxes))
        
            # image = draw.draw_roi(img=frame, tracker_results=example_tracker)
            # or
            # image = example_tracker.Drawer.draw_roi(img=frame)

            image = draw.draw_roi(img=frame, tracker_results=example_tracker)
            
            cv2.imshow('plot', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
finally:
    cap.release()
    cv2.destroyAllWindows()